In [ ]:
import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 4000


In [ ]:
ipfal17 = cobra.io.read_sbml_model("iPfal17_rewrite.xml")
print(ipfal17.objective.expression)
print(ipfal17.compartments)
print(ipfal17.annotation)
print(biomass_tabel(ipfal17))


In [ ]:
ipfal17.reactions.Lipid_prod

In [ ]:
0.2+4+35+18+1.5+4.25+0.519+14+1.5

In [ ]:
ipfal19 = cobra.io.read_sbml_model("iPfal19_with_annotation.xml")

In [ ]:
print(ipfal19.objective.expression)
print(ipfal19.compartments)
print(ipfal19.annotation)
del ipfal19

In [ ]:
ipfal21 = cobra.io.load_json_model("iPfal21.json")

In [ ]:
ipfal21.optimize()
ipfal21.summary(fva=0.95)

## Metabolic tasks as defined in the 2017 Carey et al. paper

In [ ]:
'''This script runs a series of metabolic tasks on a cobra model structure
to test wether the model is able to perform behaviors observed in
vitro or in vivo. The tasks include: KO reactions (lethal KOs observed
experimentally should be recapitulated in silico), ATP production (model
should not produce ATP if no import is permitted), purine necessity
(except hypoxathine, purines are not supplemneted in vitro, but are present in vivo, the
parasite should grow with and without purines), sugar requirement (while
it is unclear in the literature, Pf likely cannot grow unless glucose and
or fructose are present), production or import of riboflavin, nicotinamide, pyridoxine, and thiamine'''

def run_fba_without_rxns_metabolite(model, meta):
    
    rxn_set = ipfal21.metabolites.get_by_id(meta).reactions
    rxns = [rxn.id for rxn in rxn_set]
    f = None
    with model:
        model.remove_reactions(rxns)
        f = model.optimize().objective_value
    return f 

def boundary_chnge_run_fba(model, rxns, bounds):
    f = None
    with model:
        for i,reaction in enumerate(rxns):
            model.reactions.get_by_id(reaction).bounds=bounds[i] # change boundaries
        
        f = model.optimize().objective_value
    return f

lethal = 0.0
frac = 0.9*ipfal21.optimize().objective_value
#-------------------------------------------------------#
value = run_fba_without_rxns_metabolite(ipfal21, 'ribflv_c')
if value > frac:
    print('Model grows with riboflavin antimetabolite')
else:
    print(f'Passes, objective_function={value}, Ribo test')
    
value = run_fba_without_rxns_metabolite(ipfal21, 'ncam_c')
if value > frac:
    print('Model grows with nico antimetabolite')
else:
    print(f'Passes, objective_function={value}, nico test')
value = run_fba_without_rxns_metabolite(ipfal21, 'pydxn_c')
if value > frac:
    print('Model grows with pyro antimetabolite')
else:
    print(f'Passes, objective_function={value}, pyro test')
    
value = run_fba_without_rxns_metabolite(ipfal21, 'thm_c')
if value >= frac:
    print('Model grows with thiamine antimetabolite.')
else:
    print(f'Passes, objective_function={value}, thiamine test')
#--------------------------------------------------------------#

exchange_rxns = [exchange.id for exchange in ipfal21.exchanges] # find exchange reactions
with ipfal21:
    for reaction in exchange_rxns:
        ipfal21.reactions.get_by_id(reaction).bounds=(0,0) # block all exchange reactions
    ipfal21.add_boundary(ipfal21.metabolites.get_by_id('atp_c'), type='demand') # add ATP demand reaction
    ipfal21.objective_coefficient = 1 # set objective coefficient to 1
    value = ipfal21.optimize().objective_value # optimize the minimal medium for the modified model
    if value >= frac:
        print('Model produces ATP when all import reactions are blocked')
    else:
        print('Passes, no ATP production when all import reactions are blocked')
#---------------------------------------------------------------------#
# Medium composition
# Can parasite grow when hypoxanthine is the only purine imported? 
hypoxanthine =['MTAADA','ADEt','dIMP_t','INSt',
               'ADNt','GUAt','DADNt4','DGSNt',
               'DINt','GSNt','PAPt','XANt']
bound = [(-1000,0)]*len(hypoxanthine)
value = boundary_chnge_run_fba(ipfal21, hypoxanthine, bound)
if value <= lethal:
    print('Model does not grow in in vitro conditions with hypoxanthine as the sole purine')
else:
    print('Passes, model grows with hypoxanthine as the sole purine')

hypoxanthine = hypoxanthine + ['HYXNt']    
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, hypoxanthine, bound)
if value >= frac:
    print('One purine (hypoxanthine) is not necessary for growth')
else:
    print('Passes')
# What if only Adenine is available?
exrxns =['MTAADA','HYXNt','GUAt','dIMP_t',
         'INSt','ADNt','DADNt4','DGSNt',
         'DINt','GSNt','PAPt','XANt']
bound = [(-1000,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value <= lethal:
    print('Model does not grow in in vitro conditions with adenine as the sole purine')
else:
    print('Passes, Adenine test with adenine as the sole purine')
# No Adenine  
exrxns = exrxns + ['ADEt']
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print(f'Model does not grow in in vitro conditions with adenine as the sole purine')
else:
    print('Passes, adenine test b')
    
# What if only Guanine is available?  
exrxns =['MTAADA','HYXNt','ADEt','dIMP_t',
         'INSt','ADNt','DADNt4','DGSNt',
         'DINt','GSNt','PAPt','XANt']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value <= lethal:
    print(f'Model does not grow in in vitro conditions with Guanine as the sole purine')
else:
    print('Passes, Guanine test')
# No Guanine  
exrxns = exrxns + ['GUAt']
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Model does not grow in in vitro conditions with Guanine as the sole purine')
else:
    print('Passes, Guanine test b')

# What if only Inosine is available?  
exrxns =['MTAADA','HYXNt','ADEt','dIMP_t',
         'GUAt','ADNt','DADNt4','DGSNt',
         'DINt','GSNt','PAPt','XANt']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value <= lethal:
    print(f'Model does not grow in in vitro conditions with Inosine as the sole purine')
else:
    print('Passes, Inosine test')
# No Inosine  
exrxns = exrxns + ['INSt']
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Model does not grow in in vitro conditions with Inosine as the sole purine')
else:
    print('Passes, Inosine test b')
# What if only Adenosine is available?  
exrxns =['MTAADA','HYXNt','ADEt','dIMP_t',
         'GUAt','INSt','DADNt4','DGSNt',
         'DINt','GSNt','PAPt','XANt']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value <= lethal:
    print(f'Model does not grow in in vitro conditions with Adenosine as the sole purine')
else:
    print('Passes, Adenosine test')
# No Adenosine  
exrxns = exrxns + ['ADNt']
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Model does not grow in in vitro conditions with Adenosine as the sole purine')
else:
    print('Passes, Adenosine test b')
    
# What if only guanosine is available?
exrxns = ['MTAADA', 'HYXNt', 'ADEt', 'dIMP_t',
        'INSt', 'GUAt', 'DADNt4', 'DGSNt', 
        'DINt', 'ADNt', 'PAPt', 'XANt']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value <= lethal:
    print('Model does not grow in in vitro conditions with Guanosine as the sole purine')
else:
    print('Passes, Guanosine test')
# No Guanosine  
exrxns = exrxns + ['GSNt']
bound = bound + [(0,0)]
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Model does not grow in in vitro conditions with Guanosine as the sole purine')
else:
    print('Passes, Guanosine test b')
# Can model grow on glucose as only sugar source
exrxns = ['GLCt1']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Alternative sugars can replace glucose as sole sugar source (incorrectly)')
else:
    print('Passes, Glucose test')
# Can model grow without isoleucine
exrxns = ['ILELAT1tc']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print('Model grows without isoleucine')
else:
    print('Passes, isoleucine test')
# Can model growth without p-aminobenzoic acid?
exrxns = ['DHPS2']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value != lethal:
    print(f'Model grows without p-aminobenzoic acid')

# Is growth reduced without tyrosine supplementation?
exrxns = ['TYRt2r']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value < frac:
    print(f'tyrosine required for growth')
if value > frac:
    print(f'Growth is not reduced without tyrosine')

# Is growth reduced without methionine supplementation?    
exrxns = ['METt2r','METLEUex']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value < frac:
    print(f'methionine required for growth')
if value > frac:
    print(f'Growth is not reduced without methionine')

# Is growth reduced without proline supplementation?    
exrxns = ['PROt2r']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value < frac:
    print(f'proline required for growth')
if value > frac:
    print(f'Growth is not reduced without proline')
    
# Is growth reduced without glutamate supplementation?   
exrxns = ['GLUt2r']
bound = [(-1000.0,0)]*len(exrxns)
value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value < frac:
    print(f'glutamate required for growth')
if value > frac:
    print(f'Growth is not reduced without glutamate')

# Is growth reduced without glutamine supplementation? 
exrxns = ['GLNt2r']
bound = [(-1000.0,0)]*len(exrxns)
exrxns = exrxns + ['THRGLNexR','SERGLNexR','ALAGLNexR','CYSGLUexR']
bound = bound + [(0,1000)]*len(exrxns)

value = boundary_chnge_run_fba(ipfal21, exrxns, bound)
if value < frac:
    print(f'glutamine required for growth')
if value > frac:
    print(f'Growth is not reduced without glutamine')
#-----------------------------------------------------------------------------------
# Leaky metabolites
exchange_rxns = [exchange.id for exchange in ipfal21.exchanges] # find exchange reactions
leaky = np.zeros(len(exchange_rxns))
for i,rxn in enumerate(exchange_rxns):
    with ipfal21:
        for reaction in exchange_rxns:
            ipfal21.reactions.get_by_id(reaction).bounds=(0,1000) # block all exchange reactions
        ipfal21.objective = {ipfal21.reactions.get_by_id(rxn): 1}
        leaky[i] = ipfal21.optimize().objective_value
# Print leaky reactions
np.asarray(exchange_rxns)[np.where(leaky>1e-9)]

In [ ]:
ipfal21.metabolites.ribflv_c

In [ ]:
ipfal21.exchanges

In [ ]:
meta='ribflv_c'
rxn_set = ipfal21.metabolites.get_by_id(meta).reactions
rxns = [rxn.id for rxn in rxn_set]
print(rxns)

In [ ]:
['THRGLNexR','SERGLNexR','ALAGLNexR','CYSGLUexR']

In [ ]:
ipfal21.reactions.DHPS2.annotation

In [ ]:
exchange_rxns[1]

In [ ]:
print(ipfal21.objective.expression)
print(ipfal21.compartments)
print(ipfal21.annotation)
#del ipfal21

In [ ]:
def biomass_tabel(model):
    # find all substrates of the biomass reaction
    biomets = model.reactions.biomass.metabolites #r_4041 = biomassreaction 8

    s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
    m = list(biomets.keys()) # ID of the metabolite in the reaction

    # print a table of metabolites and their stoichiometry

    metname = [mi.name for mi in m]# names of the metabolite in the reaction

    # table printing (this is very messy)

    biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
    return biomass.sort_values(by='metabolite', ascending='True')

In [ ]:
# find all substrates of the biomass reaction
biomets = ipfal21.reactions.biomass.metabolites #r_4041 = biomassreaction 8

s = list(biomets.values()) #stoichiometry of the metabolites in the reaction
m = list(biomets.keys()) # ID of the metabolite in the reaction

# print a table of metabolites and their stoichiometry

metname = [mi.name for mi in m]# names of the metabolite in the reaction

# table printing (this is very messy)

biomass = pd.DataFrame.from_records(map(list, zip(*[m,metname,s])),columns=['ID', 'metabolite','stoichiometry'])
biomass.sort_values(by='metabolite', ascending='True')

In [ ]:
ipfal21.metabolites.bm_lipid_c

In [ ]:
ipfal21.metabolites.pheme_fv

In [ ]:
ipfal21.reactions.HMGLB

In [ ]:
ipfal21.reactions.lipid_bm

Lipids in biomass consist of:
1. acyl-PG 0.2
2. diacyl PG (DAG?) 4.0 -> Produced by diacyl-glycerol
3. PC 35.0
4. PE 18.0
5. PG 1.5
6. PI 4.25
7. Cholesterol 0.519
8. Sphingomylin 14.0
9. TAG 1.5  
**Interpretation**:
- Here already discrepancy with Chiappino, also how is there no PS, maybe in one of the other groups included.
- Cholesterol value very low
- as of now Chiappino seems more reasonable, but Carey model the lipid change might have higher impact

#### Questions:
 Where are these values coming from? 
 -> Gulati 2015 relative values
 
 What justification for missing PS? 
 -> PS relatively smaller in Parasite than in uRBC
1. maybe no production, but Maier data shows growth
2. export out of Parasite to iRBC membrane, thus after saponin lysis not there
3. statistical misinterpretation, PS loses out in relativ abundace as PC & PE growth much bigger, thus lower % value in total lipid content
 

In [ ]:
df_alex = pd.read_excel('/home/karnet/malaria_lipid_models/Datasets/RBC and asexual Pf lipidome.xlsx', skiprows=1)
df_alex.dropna(axis = 0, how = 'all', inplace = True)
df_alex.reset_index(drop=True, inplace=True)
df_alex.set_index('Unnamed: 0', inplace=True)

df_alex

In [ ]:
index_per_cent = [133,138,143,147,150]
df_work_alex = df_alex.iloc[133:].copy()
for i in range(4):
    phase = df_work_alex.columns[i*3][:-1]
    print(phase)
    data = df_work_alex.iloc[:,i*3:i*3+3].T
    df_work_alex['mean_'+phase] = df_work_alex.iloc[:,i*3:i*3+3].mean(axis=1)
    df_work_alex['std_'+phase] = df_work_alex.iloc[:,i*3:i*3+3].std(axis=1)
    #df_work_alex['coeff_var'+phase] = df_work_alex['std_'+phase] / df_work_alex['mean_'+phase]
df_work_alex

In [ ]:
index_work = np.asarray(index_per_cent)-133
per_cent_lipids = []
per_cent_lipids.append(df_work_alex.iloc[0:5]['mean_Trophozoite '].values[4])
per_cent_lipids.append(df_work_alex.iloc[0:5]['mean_Trophozoite '].values[2]*df_work_alex.iloc[6:10]['mean_Trophozoite '].values/100)
per_cent_lipids.append(df_work_alex.iloc[0:5]['mean_Trophozoite '].values[1]*df_work_alex.iloc[11:14]['mean_Trophozoite '].values/100)
per_cent_lipids.append(df_work_alex.iloc[0:5]['mean_Trophozoite '].values[3]*df_work_alex.iloc[15:]['mean_Trophozoite '].values/100)


In [ ]:
new_list = []

def flatten(temp_list):
    for ele in temp_list:
        if type(ele) == np.ndarray:
            flatten(ele)
        else:
            new_list.append(ele)
    
flatten(per_cent_lipids)


In [ ]:
lipids = ['chol', 'PC', 'PE','PG','PS','CE','DAG','TAG','Cer','DHSM','SM']
new_list

In [ ]:
ipfal21.genes.get_by_id('PF3D7_0906500')

In [ ]:
ipfal21.metabolites.chsterol_c.summary()

In [ ]:
ipfal21.metabolites.all_ps_c

In [ ]:
ipfal21.optimize()
ipfal21.metabolites.get_by_id('all_pe_c').summary()

In [ ]:
ipfal21.metabolites.all_pc_c.summary()

In [ ]:
normi = ipfal21.optimize().objective_value
val = 0
with ipfal21:
    ipfal21.reactions.EX_lipid_c.bounds = (0,0)
    ipfal21.optimize()
    val = ipfal21.optimize().objective_value
    print(ipfal21.metabolites.bm_lipid_c.summary())
    print(ipfal21.metabolites.chsterol_c.name)
    print(ipfal21.metabolites.chsterol_c.summary())
    print(ipfal21.metabolites.atp_c.summary())
    
print(val/normi)

In [ ]:
with ipfal21:
    ipfal21.reactions.EX_lipid_c.bounds = (0,0)
    ipfal21.optimize()
    print(ipfal21.metabolites.apg141_c.summary())
    print(ipfal21.metabolites.ptd1ino_c.summary())
    print(ipfal21.metabolites.sphmyln_hs_c.summary())
    print(ipfal21.metabolites.crm_c.summary())
    print(ipfal21.metabolites.sphmyln_c.summary())
    print(ipfal21.metabolites.dag_c.summary())
    print(ipfal21.metabolites.all_pe_c.summary())
    print(ipfal21.metabolites.get_by_id('2agpe120_c').summary())
    print(ipfal21.metabolites.get_by_id('12dgr120_c').summary())
    print(ipfal21.metabolites.pc_c.summary())
    

In [ ]:
ipfal21.reactions.CHSTEROLt.annotation

In [ ]:
ipf

In [ ]:
ipfal21.reactions.EX_lipid_c

In [ ]:
ipfal21.reactions.pe_prod20

Each lipid has a pseudo reaction where all possible side chain configurations join to produce one representative head group specific pseudo lipid.

In [ ]:
ipfal21.reactions.pe_prod1.subsystem

## Idea to access KEGG ID of lipid reactions
It would facilitate the comparison of the GEMs and model

In [ ]:
import json

In [ ]:
# Opening JSON file
f = open('iPfal21.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
# search and store for lipid reactions in dict, which have the keyword 'Lipid' in them
rxn_ids=[]
for reaction in data['reactions']:
    try:
        if 'Lipid' in reaction['subsystem']:
            rxn_ids.append(reaction['id'])
    except:
        continue
len(rxn_ids)

In [ ]:
# used pandas as easier manipulation
df = pd.DataFrame(data['reactions'])
df[df.id.isin(rxn_ids)]

In [ ]:
ipfal21.reactions.TPI.annotation['kegg.reaction']

In [ ]:
reactionstable = pd.read_csv('reactions.csv', header=0)
reactionstable

In [ ]:
#ipfarid = [reactionid[:] for reactionid in table['ID']]#
modelsrid = list(reactionstable['ID'])
#check if model ids are in iPFA reaction ids since they are shortend
#matching = [reaction for reaction in ipfarid if any(modelid in reaction for modelid in modelsrid)]
#len(matching)

In [ ]:
hypoxanthine =['MTAADA','ADEt','dIMP_t','INSt',
               'ADNt','GUAt','DADNt4','DGSNt',
               'DINt','GSNt','PAPt','XANt']
hypoxanthine = hypoxanthine + ['HYXNt']    

In [ ]:
for rxn in hypoxanthine:
    print(ipfal21.reactions.get_by_id(rxn))

In [ ]:
purines = ['HXAN','GUA','DIMP','INS','ADN','DAD_2','DGSN','DIN','GSN','PAP','XAN']
for pur in purines:
    print(ipfal21.metabolites.get_by_id(pur.lower()+'_c').annotation)